# Week 09 Handson
In this week handson, we'll try to play with Kaggle, which is one of the biggest data science community platforms. We will try to join a Kaggle competition by building a model that can predict MoA (Mechanisms of Action) in drugs development. General guide about what you need to do:
1. Register to Kaggle (if you haven't had an account yet) with your full name,
2. Download the dataset,
3. Build a model,
4. Perform an inference to the given testing data,
5. Submit the inference result

Competition link: [cick here](https://www.kaggle.com/c/lish-moa/overview)

Submission:
1. This jupyter notebook: there are at least three blocks of codes, which are data preparation, modelling and inference. However, you are free to modify, e.g., further breaking down the data prepration block to EDA and data preprocessing, etc.
2. Csv file that is submitted to the competition.
3. Screenshot of your posisition in the leaderboard (jpg file).

Zip those three files above, with a file name of "W09_your-student-id_your-name.zip" and submit to the course portal. In case the allowable size is exceeded, you can upload to, e.g., gdrive first, then upload a txt file containing that download url to the course portal. In such case, please make sure that the url is publicly open.

# Preparation

In [14]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import random
import os
# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold

import warnings
warnings.filterwarnings('ignore')

In [20]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [21]:
train_features = pd.read_csv('../../Dataset/MoA-Prediction-Competition/train_features.csv')
train_targets_scored = pd.read_csv('../../Dataset/MoA-Prediction-Competition/train_targets_scored.csv')
# train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../../Dataset/MoA-Prediction-Competition/test_features.csv')
sample_submission = pd.read_csv('../../Dataset/MoA-Prediction-Competition/sample_submission.csv')

In [22]:
train_features.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g-67

In [23]:
train_targets_scored.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [40]:
train = train_features
# train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

train = train.drop(['cp_type', 'sig_id'], axis=1)
train = pd.get_dummies(train,prefix=['cp_dose'])
target = train_targets_scored.drop(['sig_id'],axis=1)

test = test_features.drop(['cp_type', 'sig_id'], axis=1)
test = pd.get_dummies(test,prefix=['cp_dose'])

In [41]:
train

cp_time     g-0     g-1     g-2     g-3     g-4     g-5     g-6  \
0           24  1.0620  0.5577 -0.2479 -0.6208 -0.1944 -1.0120 -1.0220   
1           72  0.0743  0.4087  0.2991  0.0604  1.0190  0.5207  0.2341   
2           48  0.6280  0.5817  1.5540 -0.0764 -0.0323  1.2390  0.1715   
3           48 -0.5138 -0.2491 -0.2656  0.5288  4.0620 -0.8095 -1.9590   
4           72 -0.3254 -0.4009  0.9700  0.6919  1.4180 -0.8244 -0.2800   
...        ...     ...     ...     ...     ...     ...     ...     ...   
23809       24  0.1394 -0.0636 -0.1112 -0.5080 -0.4713  0.7201  0.5773   
23810       24 -1.3260  0.3478 -0.3743  0.9905 -0.7178  0.6621 -0.2252   
23811       48  0.3942  0.3756  0.3109 -0.7389  0.5505 -0.0159 -0.2541   
23812       24  0.6660  0.2324  0.4392  0.2044  0.8531 -0.0343  0.0323   
23813       72 -0.8598  1.0240 -0.1361  0.7952 -0.3611 -3.6750 -1.2420   

          g-7     g-8     g-9    g-10    g-11    g-12    g-13    g-14    g-15  \
0     -0.0326  0.5548 -0.0921  1.1830  0.1530  0.5574 -0.4015  0.1789 -0.6528   
1      0.3372 -0.4047  0.8507 -1.1520 -0.4201 -0.0958  0.4590  0.0803  0.2250   
2      0.2155  0.0065  1.2300 -0.4797 -0.5631 -0.0366 -1.8300  0.6057 -0.3278   
3      0.1792 -0.1321 -1.0600 -0.8269 -0.3584 -0.8511 -0.5844 -2.5690  0.8183   
4     -0.1498 -0.8789  0.8630 -0.2219 -0.5121 -0.9577  1.1750  0.2042  0.1970   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
23809  0.3055 -0.4726  0.1269  0.2531  0.1730 -0.4532 -1.0790  0.2474 -0.4550   
23810 -0.5565  0.5112  0.6727 -0.1851  2.8650 -0.2140 -0.6153  0.8362  0.5584   
23811  0.1745 -0.0340  0.4865 -0.1854  0.0716  0.1729 -0.0434  0.1542 -0.2192   
23812  0.0463  0.4299 -0.7985  0.5742  0.1421  2.2700  0.2046  0.5363 -1.7330   
23813  0.9146  3.0790  1.2460  1.9460  1.4370  2.9780  2.2370 -0.6818  0.6870   

         g-16    g-17    g-18    g-19    g-20    g-21    g-22    g-23    g-24  \
0     -0.7969  0.6342  0.1778 -0.3694 -0.5688 -1.1360 -1.1880  0.6940  0.4393   
1      0.5293  0.2839 -0.3494  0.2883  0.9449 -0.1646 -0.2657 -0.3372  0.3135   
2      0.6042 -0.3075 -0.1147 -0.0570 -0.0799 -0.8181 -1.5320  0.2307  0.4901   
3     -0.0532 -0.8554  0.1160 -2.3520  2.1200 -1.1580 -0.7191 -0.8004 -1.4670   
4      0.1244 -1.7090 -0.3543 -0.5160 -0.3330 -0.2685  0.7649  0.2057  1.3720   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
23809  0.3588  0.1600 -0.7362 -0.1103  0.8550 -0.4139  0.5541  0.2310 -0.5573   
23810 -0.2589  0.1292  0.0148  0.0949 -0.2182 -0.9235  0.0749 -1.5910 -0.8359   
23811 -0.0302 -0.4218  0.4057 -0.5372  0.1521 -0.2651  0.2310 -0.8101  0.4943   
23812  0.1450  0.6097  0.2024  0.9865 -0.7805  0.9608  0.3440  2.7650  0.4925   
23813 -1.1060  0.0182 -0.9247 -0.0738 -0.1919 -0.7722 -1.4050 -1.0050 -1.1170   

         g-25    g-26    g-27    g-28    g-29    g-30    g-31    g-32    g-33  \
0      0.2664  0.1907  0.1628 -0.2853  0.5819  0.2934 -0.5584 -0.0916 -0.3010   
1     -0.4316  0.4773  0.2075 -0.4216 -0.1161 -0.0499 -0.2627  0.9959 -0.2483   
2      0.4780 -1.3970  4.6240 -0.0437  1.2870 -1.8530  0.6069  0.4290  0.1783   
3     -0.0107 -0.8995  0.2406 -0.2479 -1.0890 -0.7575  0.0881 -2.7370  0.8745   
4      0.6835  0.8056 -0.3754 -1.2090  0.2965 -0.0712  0.6389  0.6674 -0.0783   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
23809 -0.4397 -0.9260 -0.2424 -0.6686  0.2326  0.6456  0.0136 -0.5141 -0.6320   
23810 -0.9217  0.3013  0.1716  0.0880  0.1842  0.1835  0.5436 -0.0533 -0.0491   
23811  0.6905 -0.3720 -1.4110  0.4516  1.2300 -0.1949 -1.3280 -0.4276 -0.0040   
23812  0.6698  0.2374 -0.3372  0.8771 -2.6560 -0.2000 -0.2043  0.6797 -0.0248   
23813 -0.5293 -1.1720 -0.2885  0.1599 -0.4250  0.3591 -0.1420 -0.9530 -0.2005   

         g-34    g-35    g-36    g-37    g-38    g-39    g-40    g-41    g-42  \
0     -0.1537  0.2198  0.2965 -0.5055 -0.5119 -0.2162 -0.0347 -0.2566 -1.1980   
1      0.2655 -0.2102  0.1656  0.5300

In [42]:
target

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,re

In [43]:
test

cp_time     g-0     g-1     g-2     g-3     g-4     g-5     g-6     g-7  \
0          24 -0.5458  0.1306 -0.5135  0.4408  1.5500 -0.1644 -0.2140  0.2221   
1          72 -0.1829  0.2320  1.2080 -0.4522 -0.3652 -0.3319 -1.8820  0.4022   
2          24  0.1852 -0.1404 -0.3911  0.1310 -1.4380  0.2455 -0.3390 -0.3206   
3          24  0.4828  0.1955  0.3825  0.4244 -0.5855 -1.2020  0.5998 -0.1799   
4          48 -0.3979 -1.2680  1.9130  0.2057 -0.5864 -0.0166  0.5128  0.6365   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
3977       24  0.4571 -0.5743  3.3930 -0.6202  0.8557  1.6240  0.0640 -0.6316   
3978       24 -0.5885 -0.2548  2.5850  0.3456  0.4401  0.3107 -0.7437 -0.0143   
3979       72 -0.3985 -0.1554  0.2677 -0.6813  0.0152  0.4791 -0.0166  0.7501   
3980       48 -1.0960 -1.7750 -0.3977  1.0160 -1.3350 -0.2207 -0.3611 -1.3020   
3981       72 -0.5174  0.2953  0.3286 -0.0428 -0.0800  0.8702 -0.8724  0.3883   

         g-8     g-9    g-10    g-11    g-12    g-13    g-14    g-15    g-16  \
0    -0.3260  1.9390 -0.2305 -0.3670  1.3040  1.4610  0.0043  0.6816 -0.2304   
1    -0.3528  0.1271  0.9303  0.3173 -1.0120 -0.3213  0.0607 -0.5389 -0.8030   
2     0.6944  0.5837 -0.0553 -0.6222  2.5430 -0.7857  0.8163 -0.0495  0.1806   
3     0.9365  0.2942  1.3990 -1.1080  1.1610 -0.8134  0.0743 -0.3096 -0.9956   
4     0.2611 -1.1120 -0.1747 -0.2053 -0.0006  0.0254  0.1784 -0.1314  1.0700   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
3977 -1.1990  0.7312  0.2752  0.8771  0.7721 -0.6836 -0.7374 -1.1620  0.1262   
3978  0.2615  1.9980  0.2010  0.3961  0.8776 -0.9688  0.1743  1.4400  1.6480   
3979  0.0346  1.3910  0.4557 -0.4277 -0.1186 -0.2801  0.4377 -0.7897 -0.1892   
3980  1.0150  0.6747  1.5290  0.6784  1.2500  1.3950  1.0250  0.4715 -0.1787   
3981 -0.0491  0.3557  0.2370  0.2471 -0.1459  1.5090 -0.5827  0.7675  0.6291   

        g-17    g-18    g-19    g-20    g-21    g-22    g-23    g-24    g-25  \
0    -0.0635 -0.2030 -0.6821 -0.6242  0.1297 -0.0338  0.3372  0.2254  0.4795   
1    -1.0600 -0.0978 -0.8156 -0.6514  0.6812  0.5246  0.0000  0.5030 -0.1500   
2     1.0290 -0.5204 -1.1070  0.7365 -0.3835 -0.5771  0.0523 -0.2690  0.1674   
3     1.7100  1.2200 -0.1258 -0.5900 -0.7956 -0.6765  0.0000 -0.8820  0.7609   
4     0.0477 -0.0756  0.1287  0.1403  0.7609 -0.1095  0.0000 -1.0950 -0.0780   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
3977  0.2718  0.3968 -1.1580  0.9891 -0.7642  0.6810  0.8971  0.1092  0.6508   
3978 -0.1872  0.3744  0.5423  0.6170 -0.7436 -0.1716  0.3372 -0.0668  0.3929   
3979  0.9756  0.4994 -0.9567 -1.2720 -0.1851 -0.2112  0.3372  0.8268 -0.3569   
3980  0.5192  0.0142 -0.1072  0.9399 -1.6820  0.1960 -0.7793 -1.1030  0.6484   
3981  0.4247 -0.5076 -1.7690  0.0279  0.2587 -0.8359  0.0000 -0.4299 -0.2637   

        g-26    g-27    g-28    g-29    g-30    g-31    g-32    g-33    g-34  \
0     0.7642  0.6638 -0.2480 -0.1183 -0.4847 -0.0179 -0.8204 -0.5296 -1.5070   
1    -0.1433  2.0910 -0.6556 -0.6012 -0.4104 -0.0580 -0.3608  0.2197 -0.7101   
2     0.6010 -0.6660  0.0276  0.0924  0.2785 -0.3943 -0.4602 -0.0673 -1.3420   
3    -0.0882  0.5217  0.9587 -0.4764  0.2690  0.4753  0.0196  0.2775 -0.7142   
4     0.9048 -0.3007  0.4351 -0.1558  0.2101  0.4850  0.1139  0.6745 -0.9101   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
3977 -0.9330  0.1292 -0.5895  0.4987 -0.1180  0.3460  0.6819 -0.9072  1.3150   
3978  0.1078 -0.1869  0.6547  0.5841  0.1463 -0.4880  0.7560  0.0109 -1.4160   
3979  0.4813 -0.1775 -0.0459 -0.2526 -0.3855 -0.7558 -0.3144 -0.3284  0.3375   
3980  1.0080  0.4825 -0.1814  0.4480  0.6424  0.0799 -1.9080  0.6907  0.0238   
3981 -0.2599 -0.1378 -0.7888 -0.5585 -0.3433 -0.3615 -0.8025  0.6676  0.0021   

        g-35    g-36    g-37    g-38    g-39    g-40    g-41    g-42    g-43  \
0    -0.0144  0.4448  0.5753 -0.2993 -0.9532 -0.1443 -1.0880  0.2080  

# Model

In [95]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(60, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

In [93]:
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=3, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train,validation_data=(X_test, y_test), verbose=1, epochs=10)
        # make a prediction on the test set
        yhat = model.predict(X_test)
        # round probabilities to class labels
        yhat = yhat.round()
        # calculate accuracy
        acc = accuracy_score(y_test, yhat)
        # store result
        print('>%.3f' % acc)
        results.append(acc)
    return results

In [94]:
# evaluate model
results = evaluate_model(train.to_numpy(), target.to_numpy())
# summarize performance
print('Accuracy: %.3f (%.3f)' % (np.mean(results), np.std(results)))

Epoch 1/20
497/497 [==============================] - 2s 5ms/step - loss: 0.0533 - accuracy: 0.0672 - val_loss: 0.0200 - val_accuracy: 0.0949
Epoch 2/20
497/497 [==============================] - 2s 4ms/step - loss: 0.0184 - accuracy: 0.1117 - val_loss: 0.0182 - val_accuracy: 0.1120
Epoch 3/20
497/497 [==============================] - 2s 4ms/step - loss: 0.0166 - accuracy: 0.1294 - val_loss: 0.0174 - val_accuracy: 0.1049
Epoch 4/20
497/497 [==============================] - 2s 5ms/step - loss: 0.0152 - accuracy: 0.1459 - val_loss: 0.0171 - val_accuracy: 0.1207
Epoch 5/20
497/497 [==============================] - 2s 5ms/step - loss: 0.0141 - accuracy: 0.1664 - val_loss: 0.0169 - val_accuracy: 0.1159
Epoch 6/20
497/497 [==============================] - 2s 5ms/step - loss: 0.0132 - accuracy: 0.1856 - val_loss: 0.0169 - val_accuracy: 0.1264
Epoch 7/20
497/497 [==============================] - 2s 4ms/step - loss: 0.0124 - accuracy: 0.2034 - val_loss: 0.0170 - val_accuracy: 0.1197
Epoch 

KeyboardInterrupt: 

# Training

In [46]:
new_train = train.to_numpy()
new_target = target.to_numpy()
new_test = test.to_numpy()

n_inputs, n_outputs = new_train.shape[1], new_target.shape[1]

model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(new_train, new_target, verbose=1, epochs=10)

Epoch 1/10
745/745 [==============================] - 2s 3ms/step - loss: 0.0421 - accuracy: 0.0554
Epoch 2/10
745/745 [==============================] - 3s 3ms/step - loss: 0.0179 - accuracy: 0.0987
Epoch 3/10
745/745 [==============================] - 3s 4ms/step - loss: 0.0166 - accuracy: 0.1102
Epoch 4/10
745/745 [==============================] - 3s 4ms/step - loss: 0.0159 - accuracy: 0.1172
Epoch 5/10
745/745 [==============================] - 3s 4ms/step - loss: 0.0154 - accuracy: 0.1249
Epoch 6/10
745/745 [==============================] - 3s 4ms/step - loss: 0.0149 - accuracy: 0.1343
Epoch 7/10
745/745 [==============================] - 3s 4ms/step - loss: 0.0146 - accuracy: 0.1441
Epoch 8/10
745/745 [==============================] - 3s 4ms/step - loss: 0.0142 - accuracy: 0.1529
Epoch 9/10
745/745 [==============================] - 3s 4ms/step - loss: 0.0139 - accuracy: 0.1521
Epoch 10/10
745/745 [==============================] - 3s 4ms/step - loss: 0.0137 - accuracy: 0.1584

In [50]:
prediction = model.predict(new_test[:,:])

In [73]:
def prob_clip(df, floor, ceiling):
    for c in range(df.shape[1]):
        df[:,c] = np.where(df[:,c]< floor, floor, df[:,c])
        df[:,c] = np.where(df[:,c] > ceiling, ceiling, df[:,c])
    return df

In [74]:
prediction = prob_clip(prediction, 0.00015, 0.995)

In [75]:
sample_submission

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [76]:
scored_target_cols = sample_submission.columns
scored_target_cols = scored_target_cols.drop('sig_id')
sub = sample_submission.copy()
sub[scored_target_cols] = prediction[:,:len(scored_target_cols)]

In [77]:
sub

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [79]:
sub.to_csv('submission.csv',index = False)